In [ ]:
import pandas as pd

# Load the data
data = pd.read_csv('/content/sample_data/saved_new_data.csv')

# Display the first few rows of the dataframe
data.head()

# Time series plot

In [ ]:
import matplotlib.pyplot as plt

# Assuming each column is a channel and the first column is time or index
channels = data.columns[1:]  # Adjust if your format includes a time/index column
sampling_rate = 1000  # Example: 1000 Hz, adjust as per your setup

# Plot the first few seconds of EEG data for the first few channels
plt.figure(figsize=(15, 5))
for channel in channels[:5]:  # Adjust the slice for more/less channels
    plt.plot(data[channel][:sampling_rate * 2], label=channel)  # Plotting first 2 seconds

plt.title('EEG Time Series Plot')
plt.xlabel('Time (milliseconds)')
plt.ylabel('Amplitude (uV)')
plt.legend()
plt.show()


# Spectral plot

In [ ]:
import numpy as np

# Assuming using the first channel for demonstration
fft_output = np.fft.fft(data[channels[0]][:sampling_rate])  # Use one second of data
frequencies = np.fft.fftfreq(sampling_rate, 1/sampling_rate)

# Plot the frequency spectrum
plt.figure(figsize=(10, 5))
plt.plot(frequencies[:len(frequencies)//2], np.abs(fft_output)[:len(frequencies)//2])
plt.title('Frequency Spectrum of EEG')
plt.xlabel('Frequency (Hz)')
plt.ylabel('Amplitude')
plt.show()


# Topographical Map

In [ ]:
!pip install mne

In [ ]:
# import mne

# # Assuming the first column is an index and others are EEG channels
# channels = data.columns[1:]  # Adjust if different
# sampling_rate = 1000  # Adjust this according to your data specifics

# # Create an Info object (without the montage parameter)
# info = mne.create_info(ch_names=list(channels), sfreq=sampling_rate, ch_types='eeg')

# # Create the RawArray
# raw = mne.io.RawArray(data[channels].values.T, info)  # Transpose data to fit (n_channels, n_times)

# # Define the montage (electrode positions), assuming standard 10-20 system
# montage = mne.channels.make_standard_montage('standard_1020')
# raw.set_montage(montage)  # Set the montage

# # Plotting the topographical map of EEG data
# raw.plot_psd(tmax=np.inf)  # PSD plot
# raw.plot_projs_topomap()  # Topomap plot


In [ ]:
# Example mapping: Adjust according to your actual electrode layout
channel_map = {
    '0': 'Fp1', '1': 'Fpz', '2': 'Fp2', '3': 'F7', '4': 'F3', '5': 'Fz', '6': 'F4', '7': 'F8',
    '8': 'FC5', '9': 'FC1', '10': 'FC2', '11': 'FC6', '12': 'M1', '13': 'T7', '14': 'C3', '15': 'Cz',
    '16': 'C4', '17': 'T8', '18': 'M2', '19': 'CP5', '20': 'CP1', '21': 'CP2', '22': 'CP6', '23': 'P7',
    '24': 'P3', '25': 'Pz', '26': 'P4', '27': 'P8', '28': 'POz', '29': 'O1', '30': 'Oz', '31': 'O2',
    '32': 'AF7', '33': 'AF3', '34': 'AF4', '35': 'AF8', '36': 'F5', '37': 'F1', '38': 'F2', '39': 'F6',
    '40': 'FT7', '41': 'FC3', '42': 'FC4', '43': 'FT8', '44': 'TP7', '45': 'C5', '46': 'C1', '47': 'C2',
    '48': 'C6', '49': 'TP8', '50': 'CP3', '51': 'CP4', '52': 'P5', '53': 'P1', '54': 'P2', '55': 'P6',
    '56': 'PO5', '57': 'PO3', '58': 'PO4', '59': 'PO6', '60': 'CB1', '61': 'CB2', '62': 'S1', '63': 'S2'
}

# Rename channels
data.rename(columns=channel_map, inplace=True, index=None)

In [ ]:
data.head()

In [ ]:
data.drop('Unnamed: 0', inplace=True, axis=1)

In [ ]:
data.head()

In [ ]:
import mne
# Select only the columns that are actually mapped (ignores unmapped channels)
channels_to_use = [channel for channel in channel_map.values() if channel in data.columns]

# Create an MNE Info object with the channels that we have data for
info = mne.create_info(ch_names=channels_to_use, sfreq=1000, ch_types='eeg')

# Create the RawArray with the data from the channels we are using
raw = mne.io.RawArray(data[channels_to_use].T.to_numpy(), info)

# Define the montage (electrode positions)
montage = mne.channels.make_standard_montage('standard_1020')

# Set the montage, ignoring channels that are not present in the montage
raw.set_montage(montage, on_missing='ignore')

# Visualize the data
raw.plot_psd(fmax=50)  # Showing PSD up to 50 Hz
# raw.plot_projs_topomap()  # Topomap